In [118]:
analysis_history_male = (
    ("Чахоткин Геннадий Павлович", 19880622, "Муж"),
    (
        ("Общий_анализ_крови_лейкоциты", 20240322, 7.3),
        ("Общий_анализ_крови_лейкоциты", 20240319, 8.4),
        ("Общий_анализ_крови_лейкоциты", 20240315, 9.8),
        ("Общий_анализ_крови_лейкоциты", 20240320, 8.1),
        ("Мочевина", 20240315, 6.2),
        ("Мочевина", 20240322, 4.5),
        ("Креатинин", 20240322, 105),
        ("Креатинин", 20240315, 125)
    )
)

# Такие же анализы, но для пациента без Y-хромосомы - чтобы поиграться с нормами для креатинина
analysis_history_female = (
    ("Краснухина Гвлина Сергеевна", 19670813, "Жен"),
    (
        ("Общий_анализ_крови_лейкоциты", 20240322, 10.1),
        ("Общий_анализ_крови_лейкоциты", 20240319, 8.4),
        ("Общий_анализ_крови_лейкоциты", 20240315, 8.5),
        ("Общий_анализ_крови_лейкоциты", 20240320, 8.1),
        ("Мочевина", 20240315, 6.2),
        ("Креатинин", 20240322, 105),
        ("Креатинин", 20240315, 125)
    )
)

norm_leukocytes = (4.0, 9.0) # Границы нормы лейкоцитов
norm_urea = (2.5, 8.3)
norm_creatinine = (97, 115) # Пороговые значения креатинина для 1)женщин и 2)мужчин

In [124]:
def evaluate_analysis(history, query):
    count = []
    dates = [] # Я подумал использовать отдельный список для сортировки дат
    for el in history[1]:
        if el[0] == query:
            count.append(list(el)) # Анализы превращаем из кортежей в списки для удобства манипуляции содержимого
            dates.append(el[1])

    sorted_dates = sorted(dates) # Отсортируем даты
    for i in range(len(count)):
        count[i][1] = str(count[i][1]) # Превращаем даты в строки, дабы их можно было отформатировать в читаемый формат
        sorted_dates[i] = str(sorted_dates[i]) # БЛИН а ведь можно было сделать иначе... ладно пофиг

    # Теперь на основе отсортированных дат сортируем список анализов с искомым названием:
    for i in range(len(count)-1):
        el = count[i]
        sorted_index = sorted_dates.index(el[1])
        count.insert(sorted_index, count.pop(i)) # Переставляет анализ в правильную хронологическую позицию в списке
        sorted_dates[sorted_index] = None # Как бы отмечаем, что мы уже прошли этот индекс
        
    print(f'Пациент: {history[0][0]}, {str(history[0][1])[:4]}-{str(history[0][1])[4:6]}-{str(history[0][1])[6:]}, {history[0][2]} \
          \nАнализы: "{query}"')
    for el in count:
        print(f' - {el[1][0:4]}-{el[1][4:6]}-{el[1][6:8]}: {el[2]}')

    def is_normal_leukocytes(analysis):
        if norm_leukocytes[0] <= analysis <= norm_leukocytes[1]:
            return True
        else:
            return False
        
    def is_normal_urea(analysis):
        if norm_urea[0] <= analysis <= norm_urea[1]:
            return True
        else:
            return False
        
    def is_normal_creatinine(analysis):
        if history[0][2] == 'Муж': # Проверяем на наличие Y-хромосомы - там нормы другие
            if norm_creatinine[1] <= analysis:
                return True
            else:
                return False
        else:
            if norm_creatinine[0] <= analysis:
                return True
            else:
                return False

    if len(count) > 1:
        if query == 'Общий_анализ_крови_лейкоциты':
            if is_normal_leukocytes(count[0][2]) < is_normal_leukocytes(count[-1][2]):
                print('Показатель улучшился')
            elif is_normal_leukocytes(count[0][2]) > is_normal_leukocytes(count[-1][2]):
                print('Показатель ухудшился')
            else:
                print('Без изменений')

        elif query == 'Мочевина':
            if is_normal_urea(count[0][2]) < is_normal_urea(count[-1][2]):
                print('Показатель улучшился')
            elif is_normal_urea(count[0][2]) > is_normal_urea(count[-1][2]):
                print('Показатель ухудшился')
            else:
                print('Без изменений')

        elif query == 'Креатинин':
            if is_normal_creatinine(count[0][2]) and is_normal_creatinine(count[-1][2]):
                if (count[0][2] > count[-1][2]):
                    print('Показатель улучшился')
                elif (count[0][2] == count[-1][2]):
                    print('Без изменений')
                else:
                    print('Показатель ухудшился')
            elif is_normal_creatinine(count[0][2]) < is_normal_creatinine(count[-1][2]):
                print('Показатель улучшился')
            elif is_normal_creatinine(count[0][2]) > is_normal_creatinine(count[-1][2]):
                print('Показатель ухудшился')
            elif not is_normal_creatinine(count[0][2]) and not is_normal_creatinine(count[-1][2]):
                if (count[0][2] < count[-1][2]):
                    print('Показатель улучшился')
                elif (count[0][2] == count[-1][2]):
                    print('Без изменений')
                else:
                    print('Показатель ухудшился')
    else:
        print('Недостаточно данных для оценки динамики')
    
    print('')

evaluate_analysis(analysis_history_male, "Общий_анализ_крови_лейкоциты")
evaluate_analysis(analysis_history_male, "Мочевина")
evaluate_analysis(analysis_history_female, "Общий_анализ_крови_лейкоциты")
evaluate_analysis(analysis_history_male, "Креатинин")
evaluate_analysis(analysis_history_female, "Креатинин")
evaluate_analysis(analysis_history_female, "Мочевина")

Пациент: Чахоткин Геннадий Павлович, 1988-06-22, Муж           
Анализы: "Общий_анализ_крови_лейкоциты"
 - 2024-03-15: 9.8
 - 2024-03-19: 8.4
 - 2024-03-20: 8.1
 - 2024-03-22: 7.3
Показатель улучшился

Пациент: Чахоткин Геннадий Павлович, 1988-06-22, Муж           
Анализы: "Мочевина"
 - 2024-03-15: 6.2
 - 2024-03-22: 4.5
Без изменений

Пациент: Краснухина Гвлина Сергеевна, 1967-08-13, Жен           
Анализы: "Общий_анализ_крови_лейкоциты"
 - 2024-03-15: 8.5
 - 2024-03-19: 8.4
 - 2024-03-20: 8.1
 - 2024-03-22: 10.1
Показатель ухудшился

Пациент: Чахоткин Геннадий Павлович, 1988-06-22, Муж           
Анализы: "Креатинин"
 - 2024-03-15: 125
 - 2024-03-22: 105
Показатель ухудшился

Пациент: Краснухина Гвлина Сергеевна, 1967-08-13, Жен           
Анализы: "Креатинин"
 - 2024-03-15: 125
 - 2024-03-22: 105
Показатель улучшился

Пациент: Краснухина Гвлина Сергеевна, 1967-08-13, Жен           
Анализы: "Мочевина"
 - 2024-03-15: 6.2
Недостаточно данных для оценки динамики

